In [1]:
"""
FUNDAMENTAL
TECHNICAL 
    (LINEAR_COEFFICENT)
TRADER
"""

'\nFUNDAMENTAL\nTECHNICAL \n    (LINEAR_COEFFICENT)\nTRADER\n'

In [2]:
import datetime as dt
import pandas as pd

from de.processor.dart import DART_FUNDAMENTAL_PROCESSOR
from de.processor.fdr import FDR_INFO_PROCESSOR, FDR_OHLCV_PREPROCESSOR
from de.loader.pykrx_loader import get_pykrx_loader

from de.processor.factor.fundamental import FUNDAMENTAL_FACTOR_PROCESSOR
from de.processor.factor.technical import LINEAR_COEFFICENT_FACTOR_PROCESSOR
from de.processor.factor.trader import TRADER_FACTOR_PROCESSOR

from de.model_paramter import MODEL_PARAMTER_PROCESSOR

from de.analyser.factor import FACTOR_ANALYSER
from de.processor.factor.filter import FACTOR_FILTER

In [3]:
CFG = {
    "FUNDAMENTAL": {
        "PROCESSOR": {
            "ohlcv_recent_n": 10,
        },
        "FILTER": {},
    },
    "LINEAR_COEF": {
        "PROCESSOR": {
            "factors": ["Close", "VolumeRotation"],
            "moving_average_window": 10,
            "coef_recent_n": 20,
        },
        "FILTER": {},
    },
    "TRADER": {
        "PROCESSOR": {
            "start": (dt.date.today() - dt.timedelta(days=7)).strftime("%Y-%m-%d"),
            "end": (dt.date.today() - dt.timedelta(days=1)).strftime("%Y-%m-%d"),
            # "end": dt.date.today().strftime("%Y-%m-%d"),
        },
        "FILTER": {
            "absolute_conditions": [
                "Corp > 0",
                "Foreign > 0",
                "Indivisual < 0",
            ]
        },
    },
    "FACTOR_ANALYSIS": {
        "profit_recent_n": 20,
    },
}
recen_n = CFG["FACTOR_ANALYSIS"]["profit_recent_n"]
analysis_date = (dt.date.today() - dt.timedelta(days=recen_n)).strftime("%Y-%m-%d")
CFG["analysis_date"] = analysis_date

In [4]:
# LOAD

In [5]:
# dart_fundamental
dart_fundamental_df = pd.read_csv("./de/data/dart_fundamental_df.csv", index_col=0)
dart_fundamental_processor = DART_FUNDAMENTAL_PROCESSOR(dart_fundamental_df)

In [6]:
# fdr_info
fdr_info_df = pd.read_csv("./de/data/fdr_info_df.csv", index_col=0)
fdr_info_processor = FDR_INFO_PROCESSOR(fdr_info_df)

In [7]:
# fdr_ohlcv
fdr_ohlcv_df = pd.read_csv("./de/data/fdr_ohlcv_df.csv", index_col=0)
fdr_ohlcv_preprocessor = FDR_OHLCV_PREPROCESSOR(fdr_ohlcv_df)
ohlcv_df = fdr_ohlcv_preprocessor.get_pps_fdr_ohlcv_df()

In [8]:
# split ohlcv
future_ohlcv_df = ohlcv_df[CFG["analysis_date"] < ohlcv_df["Date"]].copy()
past_ohlcv_df = ohlcv_df[ohlcv_df["Date"] < CFG["analysis_date"]].copy()

In [9]:
model_paramter_processor = MODEL_PARAMTER_PROCESSOR(CFG["FACTOR_ANALYSIS"])

In [10]:
# Fundamental

## Parameter
### 1. Get Factor_df
fundamental_factor_processor = FUNDAMENTAL_FACTOR_PROCESSOR(
    past_ohlcv_df, dart_fundamental_processor, fdr_info_processor
)
fundamental_factors_df = fundamental_factor_processor.get_fundamental_factor_df(CFG["FUNDAMENTAL"]["PROCESSOR"])

### 2. Get Parameter
fundamental_factor_analyser = FACTOR_ANALYSER(fundamental_factors_df)
fundamental_params_dict = model_paramter_processor.get_params_dict(future_ohlcv_df, fundamental_factor_analyser)
fundamental_param_df = pd.DataFrame().from_dict(fundamental_params_dict, orient="index", columns=["Value", "Variance"])

### 3. Set Paramter
fundamental_param = fundamental_param_df.nlargest(10, "Value").nsmallest(1, "Variance").index[0]
CFG["FUNDAMENTAL"]["FILTER"]["quantile_conditions"] = fundamental_param

## Model
### 1. Get Factor_df
fundamental_factor_processor = FUNDAMENTAL_FACTOR_PROCESSOR(ohlcv_df, dart_fundamental_processor, fdr_info_processor)
fundamental_factor_df = fundamental_factor_processor.get_fundamental_factor_df(CFG["FUNDAMENTAL"]["PROCESSOR"])

### 2. Get Filtered_stockcodes
factor_filter = FACTOR_FILTER(fundamental_factor_df)
fundamental_stockcodes = factor_filter.get_filtered_stockcodes(CFG["FUNDAMENTAL"]["FILTER"])

In [11]:
fundamental_param_df.nlargest(5, "Value").nsmallest(1, "Variance")

,Value,Variance
"((CEPP, 0.0, 0.3), (TLR, 0.3, 0.6))",0.051778,0.001322


In [12]:
# Technical

## Paramter
### 1. Get Factor_df
linear_coefficient_factor_processor = LINEAR_COEFFICENT_FACTOR_PROCESSOR(past_ohlcv_df, fdr_info_processor)
linear_coefficient_factors_df = linear_coefficient_factor_processor.get_linear_coef_factor_df(
    CFG["LINEAR_COEF"]["PROCESSOR"]
)

### 2. Get Parameter
linear_coefficient_factor_analyser = FACTOR_ANALYSER(linear_coefficient_factors_df)
linear_coefficient_params_dict = model_paramter_processor.get_params_dict(
    future_ohlcv_df, linear_coefficient_factor_analyser
)
linear_coefficient_param_df = pd.DataFrame().from_dict(
    linear_coefficient_params_dict, orient="index", columns=["Value", "Variance"]
)

### 3. Set Paramter
linear_coefficient_param = linear_coefficient_param_df.nlargest(10, "Value").nsmallest(1, "Variance").index[0]
CFG["LINEAR_COEF"]["FILTER"]["quantile_conditions"] = linear_coefficient_param

## Model
### 1. Get Factor_df
linear_coefficient_factor_processor = LINEAR_COEFFICENT_FACTOR_PROCESSOR(ohlcv_df, fdr_info_processor)
linear_coefficient_factors_df = linear_coefficient_factor_processor.get_linear_coef_factor_df(
    CFG["LINEAR_COEF"]["PROCESSOR"]
)

### 2. Get Filtered_stockcodes
factor_filter = FACTOR_FILTER(linear_coefficient_factors_df)
linear_coefficient_stockcodes = factor_filter.get_filtered_stockcodes(
    CFG["LINEAR_COEF"]["FILTER"], fundamental_stockcodes
)

In [13]:
linear_coefficient_param_df.nlargest(5, "Value").nsmallest(1, "Variance").index[0]

(('VolumeRotation', 0.3, 0.6), ('Close', 0.0, 0.3))

In [17]:
linear_coefficient_stockcodes

{'006260', '017390', '019680', '221980'}

In [14]:
# Trader
pykrx_loader = get_pykrx_loader()

trader_factor_processor = TRADER_FACTOR_PROCESSOR(pykrx_loader)
trader_factor_df = trader_factor_processor.get_trader_factor_df(
    linear_coefficient_stockcodes, CFG["TRADER"]["PROCESSOR"]
)

factor_filter = FACTOR_FILTER(trader_factor_df)
final_stockcodes = factor_filter.get_filtered_stockcodes(CFG["TRADER"]["FILTER"], linear_coefficient_stockcodes)

In [15]:
final_stockcodes

set()